In [19]:
import fileinput
import pyabc
import numpy as np
import scipy.stats as st
import tempfile
import os
from pyabc import ABCSMC
import pandas as pd
from pyabc import Distribution, RV
from pyabc.sampler import ConcurrentFutureSampler
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nombres_parametros=["psin0","beta","theta","tauh0","delta","gama","tauk0","f0","fk0","alfa1","alfa2","alfa3","z0","b",
"sig",
"omega",
"iva0",
"g0",
"x0",
"px0",
"pwx0",
"omegae",
"be",
"sige",
"omegad",
"bd",
"sigd",
"xt0",
"pw0",
"pwm",
"ma",
"rho0",
"rm",
"a",
"meta",
"aran0",
"tauhk",
"taukk",
"ivak",
"arank",
"psim0",
"vr0",
"ifdi0",
"ir0",
"gk",
"ppet0",
"qpet0",
"share",
"epsilon",
"rentas",
"rcc",
"oabc",
"irb0",
"rrb0",
"alfaf",
"kf",
"alfah",
"kh",
"iri0",
"sigmatc",
"deltatc",
"gamac",
"gaman",
"gamam",
"gamav",
"pib0",
"pi0",
"alfay",
"alfapi",
"kbb",
"vbb",
"rr" ,
"spread",
"spread2",
"deltav",
"kapa0",
"tauv0",
"psiv0",
"bv",
"sigv",
"omegav",
"taukv0",
"sltv",
"ppch",
"ltv0",
"chi0",
"act",
"mu",
"sdomeg0",
"rs0",
"lamdi",
"lamdv",
"rho1","rho2","rho3","rho4","rho5","rho6","rho7","rho8", "rho9", "rho10", "rho11", "rho12", "rho13" ,"rho14", "rho15", "rho16", "rho17", "rho18", "rho19", "rho20" , "rho21", "rho22", "rho23", "rho24", "rho25", "rho26", "cor1"]

In [18]:
def modificarParametro(Nombre,valor,path="/home/jose/Study/Universidad/Cuarto Semestre/DSGE economy/Modelo.mod"):
    """
    Modifica el valor del parámetro:Nombre y le asigna un valor:valor
    """
    for line in fileinput.FileInput(path,inplace=1):
        sline=line.strip().split("=")
        if sline[0].startswith((" "+nombre)):
            sline[1]=valor
    line='='.join(sline)    
    print(line)

In [ ]:
def leeResultados(path="/home/jose/Study/Universidad/Cuarto Semestre/DSGE economy/Modelo.log"):
    """
    Lee los resultados del .log, retorna una lista con los valores en orden
    """
    outputs=[]
    for line in fileinput.FileInput(path,inplace=1):
        sline=line.strip().split("	")
        if sline[0].startswith((" "+nombre)):
            outputs.append(float(sline[1]))
    return outputs

In [ ]:
datos=[2.4613,-0.753888,2.96812,0.0635072,1.89486,4.31645e-05,-2.69418,0.495181,-1.39779,3.18936,1.29643, 2.16781,
       -1.85042,-2.4155,0.167302,1.83971,-0.000158626, 2.66616,1.37835,2.45202,1.39283, 1.92379, 0.637466, 
       4.31647e-05,-1.49684e-05,-0.000629016,4.32433,4.34907,-0.000672181, 2.84693, -3.08748,-1.97177,-2.01011,
      -0.273962, -0.21428,3.28081, -2.08785, -3.56563,0.792168,0.0295998, 0.0360933,0, -0.220769,-0.0514519,-0.329729,
      -0.000158626,3.27506,3.32651,0.0965813, -0.993837,-0.501141,0.0554903, 1.92162,-1.71271,0.0632964,-1.18277e-06,
       -0.117506,0.1196,0,-0.126481,-3.03959,-0.00104043,-0.126481,2.60295,-0.000101012,0,0,-0.356675, 0.11265,-0.0158172,0,
       -2.56608,6.00366, -2.60679, -0.174442,5.44566,-12.3016,-0.0435824,-2.16282,-12.2521, -6.36253,1.98499]

In [ ]:
#Definir los priores para cada parámetro: UNIFORME en los límites dados
limits=dict(
    
psin0=(0,1),
beta=(0,1),
theta=(0,2),
tauh0=(0,0.5),
delta=(0.5,1),
gama=(1.5,2),
tauk0=(0,1),
f0=(0,1),
fk0=(0,1),
alfa1=(0,1),
alfa2=(0,1),
alfa3=(0,1),
z0=(25,30),
b=(1,2),
sig=(1,2),
omega=(0,0.5),
iva0=(0,0.5),
g0=(3.5,4),
x0=(1,2),
px0=(1.5,2.5),
pwx0=(1.5,2.5),
omegae=(1.5,2.5),
be=(-1,1),
sige=(0,0.5),
omegad=(0.5,1),
bd=(2,3),
sigd=(-2.5,-2),
xt0=(70,80),
pw0=(1.5,2.5),
pwm=(1.5,2.5),
ma=(-0.5,0.5),
rho0=(0,1),
rm=(0,1),
a=(0,1),
meta=(0,1),
aran0=(0.5,1.5),
tauhk=(1.5,2.5),
taukk=(1.5,2.5),
ivak=(1.5,2.5),
arank=(1.5,2.5),
psim0=(0,1),
vr0=(0,0.5),
ifdi0(0,1.1),
ir0=(0,1.5),
gk=(1.5,2.5),
ppet0=(0.5,1.5),
qpet0=(0.5,1),
share=(0,1),
epsilon=(10,30),
rentas=(0,2),
rcc=(0,1),
oabc=(0,0.5),
irb0=(0.5,1.5),
rrb0=(0,1),
alfaf=(1,2),
kf=(0,1),
alfah=(1,2),
kh=(0,1),
iri0=(1,2),
sigmatc=(5,15),
deltatc=(1,2),
gamac=(0,1),
gaman=(2,3),
gamam=(1.5,2),
gamav=(-1,1),
pib0=(-1,1),
pi0=(-1,1),
alfay=(-1,1),
alfapi=(-1,1),
kbb=(-1,1),
vbb=(-1,1),
rr=(-1,1),
spread=(-1,1),
spread2=(-1,1),
deltav=(-1,1),
kapa0=(-1,1),
tauv0=(-1,1),
psiv0=(-1,1),
bv=(-1,1),
sigv=(-1,1),
omegav=(-1,1),
taukv0=(-1,1),
sltv=(-1,1),
ppch=(-1,1),
ltv0=(-1,1),
chi0=(-1,1),
act=(-1,1),
mu=(-1,1),
sdomeg0=(-1,1),
rs0=(-1,1),
lamdi=(-1,1),
lamdv=(-1,1),
rho1=(-1,1),
rho2=(-1,1),
rho3=(-1,1),
rho4=(-1,1),
rho5=(-1,1),
rho6=(-1,1),
rho7=(-1,1),
rho8=(-1,1),
rho9=(-1,1),
rho10 =(-1,1),
rho11=(-1,1),
rho12 =(-1,1),
rho13 =(-1,1),
rho14=(-1,1),
rho15 =(-1,1),
rho16=(-1,1),
rho17 =(-1,1),
rho18=(-1,1),
rho19=(-1,1),
rho20 =(-1,1),
rho21=(-1,1),
rho22=(-1,1),
rho23 =(-1,1),
rho24=(-1,1),
rho25 =(-1,1),
rho26=(-1,1),
cor1=(-1,1))
prior = Distribution(**{key: RV("uniform", a, b - a)
                        for key, (a,b) in limits.items()})

In [ ]:
hiperParam=[psin0,beta,theta,tauh0,delta,gama,tauk0,f0,fk0,alfa1,alfa2,alfa3,z0,b,sig,omega,iva0,g0,x0,px0,
            pwx0,omegae,be,sige,omegad,bd,sigd,xt0,pw0,pwm,ma,rho0,rm,a,meta,aran0,tauhk,taukk,ivak,arank,psim0,vr0,ifdi0,
            ir0,gk,ppet0,qpet0,share,epsilon,rentas,rcc,oabc,irb0,rrb0,alfaf,kf,alfah,kh,iri0,sigmatc, eltatc,gamac,gaman,
            gamam,gamav,pib0,pi0,alfay,alfapi,kbb,vbb,rr,spread,spread2,deltav,kapa0,tauv0,psiv0,bv,sigv,omegav,taukv0,sltv,
            ppch,ltv0,chi0,act,mu,sdomeg0,rs0,lamdi,lamdv,rho1,rho2,rho3,rho4,rho5,rho6,rho7,rho8,rho9,rho10, rho11,rho12 ,
            rho13 ,rho14,rho15 ,rho16,rho17 ,ho18,rho19,rho20 ,rho21,rho22,rho23 ,rho24,rho25 ,rho26,cor1]

In [ ]:
nombres_resul=[
"c",
"n",
"w" ,   
"ir" ,    
"bf"  , 
"er",
"q" ,
"i"     ,
"ifdi"   ,
"k",
"kfdi",
"kv",
"r",
"rfdi",
"fi",
"df",
"py",
"y",
"g",
"d",
"m",
"s",
"x",
"pm",
"pd",
"px",
"xx",
"xt",
"pwx",
"pib",
"ri",
"tauh,"
"tauk",
"f",
"fk",
"z",
"iva,"
"aran",
"dm",
"pi",
"vr",
"ppet",
"qpet",
"lamda",
"ga",
"po",
"v1",
"v2",
"iri",
"gb",
"rb",
"irb",
"tc",
"cb",
"ira",
"walras",
"scc",
"sck",
"pw",
"iv",
"rv",
"pv",
"yv",
"yt",
"pt",
"taukv",
"kapa",
"ltv" ,
"irh"   ,
"ch" ,
"tauv",
"rrb",
"psiv",
"psim" ,
"psin" ,
"u"   ,
"pomeg", 
"omeg",
"rs",
"sdomeg" ,
"chi"   ,
"ct"

]

In [ ]:
#########################################
#          Definicion del Modelo        #
#########################################
def modelo(parametros):
    for i in range(len(nombres_parametros)):
        modificarParametro(nombres_parametros[i],parametros[i],path="/home/jose/Study/Universidad/Cuarto Semestre/DSGE economy/Modelo.mod")
    #Se debe indicar bien la forma correcta de llamar la finción de Matlab
    os.system("cd /usr/local/MATLAB/R2020b;")   
    os.system("matlab -nodisplay -nosplash -nodesktop -r \"run('/home/jose/Study/Universidad/Cuarto Semestre/DSGE economy/Modelo.m');exit;\" | tail -n +11")  
    output=leeResultados(path="/home/jose/Study/Universidad/Cuarto Semestre/DSGE economy/Modelo.log")
    diccionario={}
    for i in range(len(nombres_resul)):
        diccionario[nombres_resul[i]]=output[i]

In [ ]:
def distance(x, y):
    return abs(x["data"] - y["data"])
abc = pyabc.ABCSMC(model, prior, distance)

In [ ]:

acceptor = pyabc.StochasticAcceptor()
kernel = pyabc.IndependentNormalKernel(var=sigma**2)
eps = pyabc.Temperature()

abc = pyabc.ABCSMC(model, prior, None, eps=eps, acceptor=acceptor)
abc.new(pyabc.create_sqlite_db_id(), datos)
history_acceptor = abc.run(max_nr_populations=1)

In [ ]:

abc.new("~/test")
history = abc.run(max_nr_populations=1, minimum_epsilon=0.001)

In [ ]:

from pyabc.visualization import plot_kde_matrix

df, w = history.get_distribution(m=0)
plot_kde_matrix(df, w, limits=limits)